In [33]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [34]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [35]:
train['DetectedCamera'].value_counts()

Front    10910
Right    10516
Left      9298
Rear      7761
Name: DetectedCamera, dtype: int64

In [36]:
#encode as integer
mapping = {'Front':0, 'Right':1, 'Left':2, 'Rear':3}
train = train.replace({'DetectedCamera':mapping})
test = test.replace({'DetectedCamera':mapping})

In [37]:
#renaming column
train.rename(columns = {'SignFacing (Target)': 'Target'}, inplace=True)

In [38]:
#encode Target Variable based on sample submission file
mapping = {'Front':0, 'Left':1, 'Rear':2, 'Right':3}
train = train.replace({'Target':mapping})

In [39]:
#target variable
y_train = train['Target']
test_id = test['Id']

In [40]:
#drop columns
train.drop(['Target','Id'], inplace=True, axis=1)
test.drop('Id',inplace=True,axis=1)

In [41]:
train.head()

,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,3,195,1.02,46,45
1,3,203,1.09,59,54
2,0,26,0.96,104,108
3,3,199,0.81,38,47
4,3,208,0.93,54,58


In [42]:
train.columns

Index(['DetectedCamera', 'AngleOfSign', 'SignAspectRatio', 'SignWidth',
       'SignHeight'],
      dtype='object')

In [43]:
features = ['AngleOfSign', 'SignAspectRatio', 'SignWidth']

In [24]:
from sklearn import cross_validation,metrics
from sklearn.grid_search import GridSearchCV

In [55]:
param_test = {
    'max_depth':[5,10,15,20,25],
    'min_samples_split':[2,10,20,50,100]
}
gsearch = GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, max_depth=2,min_samples_split=2,
                                                          min_samples_leaf=1,n_jobs=-1,random_state=111),
                        param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'max_depth': 15, 'min_samples_split': 20}, 0.96047739206776639)

In [56]:
param_test = {
    'max_depth':list(range(11,21,1)),
    'min_samples_split':[11,13,15,17,19,20,30]
}
gsearch = GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, max_depth=2,min_samples_split=2,
                                                          min_samples_leaf=1,n_jobs=-1,random_state=111),
                        param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'max_depth': 13, 'min_samples_split': 30}, 0.96076313340612174)

In [57]:
param_test = {
    'min_samples_split':[20,30,40,50]
}
gsearch = GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, max_depth=13,min_samples_split=2,
                                                          min_samples_leaf=1,n_jobs=-1,random_state=111),
                        param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'min_samples_split': 30}, 0.96076313340612174)

In [59]:
param_test = {
    'min_samples_split':[25,31,32,33,34,35,36,37,38,39,40]
}
gsearch = GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, max_depth=13,min_samples_split=2,
                                                          min_samples_leaf=1,n_jobs=-1,random_state=111),
                        param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'min_samples_split': 33}, 0.96097105385176607)

In [60]:
param_test = {
    'min_samples_leaf':[1,2,5,10,15]
}
gsearch = GridSearchCV(estimator = RandomForestClassifier(n_estimators=100, max_depth=13,min_samples_split=33,
                                                          min_samples_leaf=1,n_jobs=-1,random_state=111),
                        param_grid = param_test,n_jobs=4,iid=False, cv=10)
gsearch.fit(X,y)
gsearch.best_params_, gsearch.best_score_

({'min_samples_leaf': 1}, 0.96097105385176607)

In [44]:
#train model
clf =  RandomForestClassifier(n_estimators=1000, max_depth=13,min_samples_split=33,
                            min_samples_leaf=1,n_jobs=-1,random_state=111,oob_score=True)
clf.fit(train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=33,
            min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=-1,
            oob_score=True, random_state=111, verbose=0, warm_start=False)

In [45]:
#predict on test data
pred = clf.predict_proba(test[features])

ValueError: Number of features of the model must  match the input. Model n_features is 5 and  input n_features is 3 

In [ ]:
#write submission file and submit
columns = ['Front','Left','Rear','Right']
sub = pd.DataFrame(data=pred, columns=columns)
sub['Id'] = test_id
sub = sub[['Id','Front','Left','Rear','Right']]
sub.to_csv("sub_rfih.csv", index=False) #99.8XXX